---
# Spatial Pooler
- SDR (Sparse Destribution Representation, 희소 분포 표상) 생성
- 입력에 대해 pooler 를 이루고 있는 column 들의 일부 (=~2% 내외) 만 활성화 한다
- 모든 column 이 비슷한 빈도로 active 되어야함
- 모든 column 의 synapse 들이 비슷한 빈도로 active 되어야함


- [x] 1-d array column structure
- [x] potential synapse - make tunable, randomly
- [ ] hard testing
- [ ] visualizing method
- [x] 학습 기능 on/off 구현
- [ ] boost factor 목표치
- [ ] activate duty 목표치 안정화

# SDR 의 조건
- The first property of the SP is to form fixed-sparsity representations of the input.
- A second desirable property is that the system should utilize all available resources to learn optimal representations of the inputs.
- A third desirable property is that output representations should be robust to noise in the inputs.
- A fourth property is that the system should be flexible and able to adapt to changing input statistics.
- Finally, a fifth property is that the system should be fault tolerant.

---

> col : 128, in_size : 32, threshold : 3, perm : 0.8, w : 10, val : 0-30 일 때 안정적인듯. 비슷한 입력에 대해서 유사한 SDR 공간을 공유하고 sparsity 가 일정했고 overlap duty 동 적당했다. 이유는?
---

In [1]:
import numpy as np
import collections
import matplotlib.pyplot as plt
import math
import nbimporter
import Encoder
import random
import time

Importing Jupyter notebook from Encoder.ipynb


In [2]:
class SpatialPooler:
    def __init__(self, input_size, columns=100, conPerm=.5, minOver=5, potential_rate=.8):
        self.input_size = input_size                               # input vector 크기
        self.input_data = np.empty([self.input_size])
        self.columnCount = columns                                 # column 의 가로 크기
        self.connectedPerm = conPerm                               # synapse 활성화(1) 될 permanence 임계치
        self.min_overlap = minOver                                 # 발화 하기 위한 컬럼 당 최소한의 overlap count
        self.minGlobalActivity = 0                                 # 승리하기 위해 필요한 score (global inhibition)
        self.desiredGlobalActivity = int(0.02 * self.columnCount)  # 한 번에 승리할 column 수 (global inhibition)
        self.minDutyCycle = 0                                      # column 당 최소 발화 duty
        self.highDutyCycle = 0
        self.permanence_inc = .05                                  # 학습시 permanence 증가량
        self.permanence_dec = .05                                 # 학습시 permanence 감소량
        self.history_capacity = 200
        self.step = 0                                              # 데이터 처리한 수
        self.potential_rate = potential_rate                       # 입력 데이터에 대한potential synapses 의 비율
        self.potential_count = int(self.input_size*self.potential_rate)
        
        self.potential_synapses = np.random.random([self.input_size, self.columnCount])   # potential synapses(proxiaml dendrite) - permanence ndarry. 초기화 필요      
        self.connected_synapses = np.zeros([self.input_size, self.columnCount])           # 연결된 synapses
        self.boosts = np.ones([self.columnCount])                                             # 보정에 필요한 boost 
        self.overlapped = np.zeros([self.columnCount])                                        # input 과 연결된 synapse 들과의 최초 계산
        self.activeColumns = np.zeros([self.columnCount])
        self.activeHistory = []                                            # active duty 를 계산하기 위한 active 기록
        self.overlapHistory = []                                           # overlap duty 를 계산하기 위한 overlap 기록
        self.activeDutyInfo = np.zeros([self.columnCount])                 # active duty 정보
        self.overlapDutyInfo = np.zeros([self.columnCount])                # overlap duty 정보
        
        # potential synapses 구성
        
        for col in range(self.columnCount):
            never_connect = self.input_size - self.potential_count

            while(never_connect > 0):
                idx = random.randint(0, self.input_size - 1)

                if(self.potential_synapses[idx, col] != -1):
                    self.potential_synapses[idx, col] = -1
                    never_connect -= 1
        
        ## duty 계산을 위한 history 생성 ##
        for c in range(self.columnCount):
            self.activeHistory.append(collections.deque())
            self.overlapHistory.append(collections.deque())
            
                
    ''' SDR 생성 '''
    def compute_SDR(self, input_data, learn=True):
        
        self.input_data = input_data
        
        ## 1. overlaping ##
        self.connected_synapses = self.potential_synapses > self.connectedPerm
        self.overlapped = self.input_data @ self.connected_synapses
        
        for c in range(self.columnCount):
            if(self.overlapped[c] > self.min_overlap):
                self.overlapped[c] *= self.boosts[c]

                if(len(self.overlapHistory[c]) >= self.history_capacity):
                    self.overlapHistory[c].popleft()

                self.overlapHistory[c].append(True)

            else:
                self.overlapped[c] = 0

                if(len(self.overlapHistory[c]) >= self.history_capacity):
                    self.overlapHistory[c].popleft()

                self.overlapHistory[c].append(False)
                    
                    
        ## 2. inhibition (global) ##
        self.minGlobalActivity = self.kthScore(self.desiredGlobalActivity)
        self.activeColumns = self.overlapped >= self.minGlobalActivity
        
        for c in range(self.columnCount):                
            if(len(self.activeHistory[c]) >= self.history_capacity):
                self.activeHistory[c].popleft()

            self.activeHistory[c].append(self.activeColumns[c])
                    
                
        ## 3. learning ## 
        if(learn):
            for c in range(self.columnCount):

                if(self.activeColumns[c] == 1):
                    for s in range(self.input_size):

                        if(self.potential_synapses[s, c] != -1):
                            if(self.input_data[s] == 1):
                                self.potential_synapses[s, c] += self.permanence_inc
                                self.potential_synapses[s, c] = min(self.potential_synapses[s, c], 1.0)
                            else:
                                self.potential_synapses[s, c] -= self.permanence_dec
                                self.potential_synapses[s, c] = max(0.0, self.potential_synapses[s, c])


        self.update_activeDuty()
        self.update_overlapDuty()
        self.step += 1
        
        ## 3.2. 보정 작업 ##
        if(learn):
            for c in range(self.columnCount):
                ## 자주 승리하지 못하는 column 에 대하여 잘 발화할 수 있도록 boost 시켜줌
                maxDuty, highDuty = self.maxhighDutyCycle()
                self.minDutyCycle = .01 * maxDuty
                self.highDutyCycle = highDuty
                #print("min :", self.minDutyCycle)
                self.boostFunction(c)
                #print(self.boosts)

                ## input 과 잘 겹치지 않는 synapse 에 대해서 permanence 증가시켜줌
                if(self.overlapDutyInfo[c] < self.minDutyCycle):
                    self.increase_Permanence(c)
                    #print("min", self.minDutyCycle)

                
# ------------------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------------------ #


    def getActiveColumns(self):
        return self.activeColumns
                        
                        
    ''' global 하게 승리할 컬럼의 기준 '''
    def kthScore(self, desired_kth):
        
        rank = self.overlapped.ravel().copy()
        rank.sort()        
        score = rank[-desired_kth]
        
        return score
    
    
    ''' global 하게 가장 자주 승리한 컬럼의 duty '''
    def maxhighDutyCycle(self):
        
        rank = self.activeDutyInfo.ravel().copy()
        rank.sort()
        maxDuty = rank[-1]
        highDuty = rank[-int(self.input_size/5)]
        
        return maxDuty, highDuty
    
    
    
    ''' 해당 column 이 발화하도록 격려 '''
    def boostFunction(self, c):
        
        x = self.activeDutyInfo[c]
        
        if(x > .02):
            self.boosts[c] = -5*x + 1.1
        else:
            self.boosts[c] = -75*x + 2.5
            
            
            
    ''' 해당 column 의 모든 petential synapse 의 permanence 를 증가시켜 잘 겹치도록 격려 '''
    def increase_Permanence(self, c):
        for s in range(self.input_size):
            if(self.potential_synapses[s, c] != -1):
                self.potential_synapses[s, c] += self.permanence_inc
        
        
    ''' activeDuty update '''
    def update_activeDuty(self):
        for c in range(self.columnCount):
            self.activeDutyInfo[c] = np.mean(self.activeHistory[c])

                
    ''' overlapDuty update '''
    def update_overlapDuty(self):
        for c in range(self.columnCount):
            self.overlapDutyInfo[c] = np.mean(self.overlapHistory[c])
    
    def visualize_SDR(self):
        fig = plt.figure(figsize=(20,1))
        plt.imshow(self.activeColumns.reshape(1, self.columnCount))
        #plt.subplots_adjust(bottom=0.1, right=0.8, top=0.9)
        #cax = plt.axes([0.85, 0.1, 0.075, 0.8])
        #plt.colorbar(cax=cax)
        #plt.show()
        
        sparsity = (np.count_nonzero(self.activeColumns==True)/(self.columnCount))
        return sparsity
    
    def test(self):
        idx = []
        for i in range(len(self.activeColumns)):
            if(self.activeColumns[i] == 1):
                idx.append(i)
                
        return idx

In [60]:
sp = SpatialPooler(128, 512, .7, 3, potential_rate=.6)
se = Encoder.ScalarEncoder(out_size=128, max_val=30, w=20)

In [65]:
step = 200
encoded_data = 0

start = time.time()
for i in range(step):
    rand = random.randint(0, se.max_val)
    encoded_data = se.encode(rand)
    sp.compute_SDR(encoded_data)
    
print(time.time() - start)

15.503398180007935


In [66]:
for i in range(1, se.max_val):
    sp.compute_SDR(se.encode(i), False)
    #sp.visualize_SDR()
    print(sp.test())
    
#sp.compute_SDR(se.encode(3))
#sp.visualize_SDR()

print("boost : \n", sp.boosts)
print("-"*50 + '\n')
print("-> 평균 : {}, 표준편차 : {}\n".format(sp.boosts.mean(), sp.boosts.std()))
print("active : \n", sp.activeDutyInfo)
print("-"*50 + '\n')
print("-> 평균 : {}, 표준편차 : {}\n".format(sp.activeDutyInfo.mean(), sp.activeDutyInfo.std()))
print("overlap : \n", sp.overlapDutyInfo)
print("-"*50 + '\n')
print("-> 평균 : {}, 표준편차 : {}\n".format(sp.overlapDutyInfo.mean(), sp.overlapDutyInfo.std()))

[106, 117, 130, 153, 193, 217, 385, 446, 467, 479, 480, 493]
[0, 30, 175, 255, 290, 294, 300, 319, 324, 353, 505]
[30, 38, 157, 255, 290, 294, 300, 324, 353, 378, 404, 505]
[5, 8, 25, 84, 115, 175, 393, 395, 454, 486]
[5, 8, 25, 65, 84, 115, 139, 159, 451, 509]
[5, 15, 25, 65, 84, 139, 159, 408, 486, 509]
[15, 66, 139, 159, 204, 311, 408, 486, 499, 509]
[14, 25, 53, 59, 61, 74, 136, 213, 215, 265, 465, 497, 509]
[14, 59, 61, 102, 214, 238, 247, 265, 455, 459]
[102, 145, 183, 184, 185, 214, 238, 320, 343, 423, 462]
[102, 145, 184, 185, 214, 238, 246, 320, 423, 428, 430]
[57, 61, 155, 169, 185, 232, 255, 421, 423, 430]
[37, 38, 49, 61, 169, 258, 304, 402, 411, 470]
[37, 42, 79, 121, 245, 278, 293, 364, 399, 419, 424, 470]
[67, 239, 245, 278, 291, 296, 301, 316, 355, 390, 410]
[47, 51, 55, 79, 90, 131, 276, 293, 405, 410, 414, 424]
[41, 51, 133, 255, 276, 372, 399, 405, 414, 487]
[51, 112, 133, 189, 237, 255, 276, 338, 386, 484]
[22, 51, 112, 133, 195, 252, 255, 314, 386, 456]
[40, 44, 54